## Betfair

In [104]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
import datetime as dt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
from src.bets.api import BetsAPI
from betfairlightweight import APIClient
uname = os.environ['betfair_username']
pword = os.environ['betfair_password']
key = os.environ['betfair_key']
bf = APIClient(uname, pword, app_key=key,
               certs='/Users/jonaspaulwestermann/Dev/marchmadness/certs',
               locale='spain', lightweight=True)
bets = BetsAPI(bf, league_name='NCAAB')
bets.teams.sample(2)

StopIteration: 
                    It seems like the requested league with name 'NCAAB'
                    is currently not available as no matching league
                    can be found on Betfair
                    Available leagues (competitions) are: ['Swiss Challenge League', 'Brazilian Mineiro', 'Brazilian Cearense', 'FIBA Champions League', 'US Major League Soccer', 'Bulgarian A League', 'CONCACAF Champions League', 'Danish Superliga', 'Vietnamese U19', 'Dutch Eredivisie', 'Torneo Federal A', 'English League 1', 'Turkish Super League', 'English League 2', 'FIBA Europe Cup Men', 'Colombian Primera A', 'Romanian Liga I', 'Brazilian Gaucho', 'Indian Wells 2018', 'Copa Sudamericana', 'Norwegian Eliteserien', 'Brazilian Catarinense', 'French Ligue 1', 'NBA', 'French Ligue 2', 'Argentinian Primera Division', 'German Bundesliga 1', 'English Premier League', 'German Bundesliga 2', 'Euroleague Women', 'Turkish 1 Lig', 'Greek Super League', 'Russian Division 1', 'Premier League Cup',

In [101]:
bets.market_catalogue.runners.values[0]

[{'handicap': 0.0,
  'metadata': {'runnerId': '5851482'},
  'runnerName': 'Under 0.5 Goals',
  'selectionId': 5851482,
  'sortPriority': 1},
 {'handicap': 0.0,
  'metadata': {'runnerId': '5851483'},
  'runnerName': 'Over 0.5 Goals',
  'selectionId': 5851483,
  'sortPriority': 2}]

In [32]:
ncaa_teams = 
ncaa_teams['runnerId'] = ncaa_teams.metadata.apply(lambda d: d['runnerId'])
ncaa_teams.drop('metadata', axis=1, inplace=True)
ncaa_teams.set_index('runnerId', inplace=True)
print(ncaa_teams.shape)
ncaa_teams.sample()

(16, 4)


,handicap,runnerName,selectionId,sortPriority
runnerId,,,,
721825,0,Alabama State,721825,1


In [34]:
ncaa_market = pd.merge(ncaa_market_catalogue, ncaa_market_book,
                       left_index=True, right_index=True,
                       suffixes=['_catalogue','_book'])

In [36]:
ncaa_market.sample().squeeze().runners_book

[{'ex': {'availableToBack': [{'price': 3.6, 'size': 39.04}],
   'availableToLay': [{'price': 4.6, 'size': 30.5},
    {'price': 4.7, 'size': 30.0}],
   'tradedVolume': []},
  'handicap': 0.0,
  'selectionId': 5145582,
  'status': 'ACTIVE',
  'totalMatched': 0.0},
 {'ex': {'availableToBack': [{'price': 1.28, 'size': 109.38},
    {'price': 1.27, 'size': 110.24}],
   'availableToLay': [{'price': 1.38, 'size': 101.5}],
   'tradedVolume': []},
  'handicap': 0.0,
  'selectionId': 2909529,
  'status': 'ACTIVE',
  'totalMatched': 0.0}]